# WIP: Native Support for Units

* Allow use of units in basic processing (e.g. column names, preserve unit state, addition/subtraction etc)
* Allow switching of units for indivudal columns 
* Allow output with units attached for e.g. CSV export 
* Propagate units naturally for e.g. normalization

In [1]:
import pandas as pd
import numpy as np
import pint
import pint_pandas
from pyrolite.util.synthetic import normal_frame
from pyrolite.geochem.ind import REE

In [2]:
df = normal_frame(
    columns=["MgO_wt%", "FeO_wt%", "Al2O3_wt%", "Ti_ppm"] + [c + "_ppm" for c in REE()]
)
df *= np.array([[100 if "wt%" in c else 10000 for c in df.columns]])
df = df.round(
    pd.Series(np.array([2 if "wt%" in c else 0 for c in df.columns]), index=df.columns)
)
df

,MgO_wt%,FeO_wt%,Al2O3_wt%,Ti_ppm,La_ppm,Ce_ppm,Pr_ppm,Nd_ppm,Sm_ppm,Eu_ppm,Gd_ppm,Tb_ppm,Dy_ppm,Ho_ppm,Er_ppm,Tm_ppm,Yb_ppm,Lu_ppm
0,1.30,12.13,0.60,319.0,386.0,2024.0,381.0,1207.0,371.0,71.0,1094.0,154.0,222.0,555.0,135.0,854.0,460.0,364.0
1,1.21,13.14,0.63,324.0,425.0,1897.0,391.0,1144.0,371.0,65.0,1078.0,136.0,228.0,609.0,132.0,876.0,466.0,360.0
2,1.44,11.87,0.65,351.0,404.0,1744.0,386.0,1315.0,362.0,74.0,1078.0,140.0,226.0,593.0,134.0,960.0,465.0,373.0
3,1.29,12.70,0.65,326.0,398.0,1880.0,375.0,1209.0,362.0,78.0,1067.0,130.0,235.0,623.0,119.0,884.0,481.0,369.0
4,1.24,11.72,0.65,365.0,433.0,1777.0,385.0,1106.0,389.0,82.0,1149.0,141.0,230.0,632.0,143.0,958.0,475.0,374.0
5,1.22,13.68,0.56,300.0,357.0,1940.0,371.0,1394.0,349.0,66.0,937.0,126.0,198.0,551.0,129.0,963.0,429.0,346.0
6,1.20,12.93,0.61,328.0,428.0,1831.0,375.0,1291.0,370.0,78.0,1036.0,149.0,236.0,558.0,122.0,881.0,475.0,367.0
7,1.30,11.91,0.65,330.0,403.0,2002.0,388.0,1204.0,370.0,54.0,1031.0,139.0,225.0,665.0,141.0,813.0,488.0,361.0
8,1.16,12.67,0.51,305.0,364.0,2120.0,352.0,1342.0,355.0,74.0,985.0,138.0,214.0,533.0,118.0,874.0,443.0,350.0
9,1.05,11.75,0.68,323.0,356.0,2408.0,372.0,1210.0,366.0,60.0,893.0,131.0,216.0,599.0,124.0,758.0,483.0,352.0


## Internalizing Units from Column Names

In [3]:
import re
from pyrolite.geochem.parse import tochem


UNIT_PATTERN = r"^(?P<analyte>[A-Za-z0-9]+)_(?P<unit>.+)$"


def internalize_units(
    df,
    unit_pattern=UNIT_PATTERN,
    unit_mapping={"wt%": "%"},
):
    if isinstance(unit_pattern, str):
        unit_pattern = re.compile(unit_pattern)
    if unit_mapping is None:
        unit_mapping = {}

    matches = pd.DataFrame({c: unit_pattern.match(c).groupdict() for c in df.columns}).T
    matches["unit"] = matches["unit"].map(lambda x: unit_mapping.get(x, x))

    return pd.DataFrame(
        {
            v.get("analyte") if v.get("analyte") is not None else k: pd.Series(
                df[k].values, dtype="pint[{}]".format(matches.loc[k, "unit"])
            )
            for k, v in matches.iterrows()
        }
    )

In [4]:
units_df = internalize_units(df)
units_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype        
---  ------  --------------  -----        
 0   MgO     10 non-null     pint[percent]
 1   FeO     10 non-null     pint[percent]
 2   Al2O3   10 non-null     pint[percent]
 3   Ti      10 non-null     pint[ppm]    
 4   La      10 non-null     pint[ppm]    
 5   Ce      10 non-null     pint[ppm]    
 6   Pr      10 non-null     pint[ppm]    
 7   Nd      10 non-null     pint[ppm]    
 8   Sm      10 non-null     pint[ppm]    
 9   Eu      10 non-null     pint[ppm]    
 10  Gd      10 non-null     pint[ppm]    
 11  Tb      10 non-null     pint[ppm]    
 12  Dy      10 non-null     pint[ppm]    
 13  Ho      10 non-null     pint[ppm]    
 14  Er      10 non-null     pint[ppm]    
 15  Tm      10 non-null     pint[ppm]    
 16  Yb      10 non-null     pint[ppm]    
 17  Lu      10 non-null     pint[ppm]    
dtypes: pint[percent](3), pint[ppm](15

Built in unit-conversion:

In [5]:
units_df.astype("pint[ppm]").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype    
---  ------  --------------  -----    
 0   MgO     10 non-null     pint[ppm]
 1   FeO     10 non-null     pint[ppm]
 2   Al2O3   10 non-null     pint[ppm]
 3   Ti      10 non-null     pint[ppm]
 4   La      10 non-null     pint[ppm]
 5   Ce      10 non-null     pint[ppm]
 6   Pr      10 non-null     pint[ppm]
 7   Nd      10 non-null     pint[ppm]
 8   Sm      10 non-null     pint[ppm]
 9   Eu      10 non-null     pint[ppm]
 10  Gd      10 non-null     pint[ppm]
 11  Tb      10 non-null     pint[ppm]
 12  Dy      10 non-null     pint[ppm]
 13  Ho      10 non-null     pint[ppm]
 14  Er      10 non-null     pint[ppm]
 15  Tm      10 non-null     pint[ppm]
 16  Yb      10 non-null     pint[ppm]
 17  Lu      10 non-null     pint[ppm]
dtypes: pint[ppm](18)
memory usage: 1.5 KB


In [6]:
df.iloc[:5, :5]

,MgO_wt%,FeO_wt%,Al2O3_wt%,Ti_ppm,La_ppm
0,1.30,12.13,0.60,319.0,386.0
1,1.21,13.14,0.63,324.0,425.0
2,1.44,11.87,0.65,351.0,404.0
3,1.29,12.70,0.65,326.0,398.0
4,1.24,11.72,0.65,365.0,433.0


In [7]:
units_df.iloc[:5, :5]

,MgO,FeO,Al2O3,Ti,La
0,1.3,12.13,0.6,319.0,386.0
1,1.21,13.14,0.63,324.0,425.0
2,1.44,11.87,0.65,351.0,404.0
3,1.29,12.7,0.65,326.0,398.0
4,1.24,11.72,0.65,365.0,433.0


In [8]:
units_df.iloc[:5, :5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype        
---  ------  --------------  -----        
 0   MgO     5 non-null      pint[percent]
 1   FeO     5 non-null      pint[percent]
 2   Al2O3   5 non-null      pint[percent]
 3   Ti      5 non-null      pint[ppm]    
 4   La      5 non-null      pint[ppm]    
dtypes: pint[percent](3), pint[ppm](2)
memory usage: 328.0 bytes


Easy reincoporation of units:

In [9]:
units_df.pint.dequantify()  # .pint.quantify() can be used for the inverse

,MgO,FeO,Al2O3,Ti,La,Ce,Pr,Nd,Sm,Eu,Gd,Tb,Dy,Ho,Er,Tm,Yb,Lu
unit,percent,percent,percent,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm,ppm
0,1.30,12.13,0.60,319.0,386.0,2024.0,381.0,1207.0,371.0,71.0,1094.0,154.0,222.0,555.0,135.0,854.0,460.0,364.0
1,1.21,13.14,0.63,324.0,425.0,1897.0,391.0,1144.0,371.0,65.0,1078.0,136.0,228.0,609.0,132.0,876.0,466.0,360.0
2,1.44,11.87,0.65,351.0,404.0,1744.0,386.0,1315.0,362.0,74.0,1078.0,140.0,226.0,593.0,134.0,960.0,465.0,373.0
3,1.29,12.70,0.65,326.0,398.0,1880.0,375.0,1209.0,362.0,78.0,1067.0,130.0,235.0,623.0,119.0,884.0,481.0,369.0
4,1.24,11.72,0.65,365.0,433.0,1777.0,385.0,1106.0,389.0,82.0,1149.0,141.0,230.0,632.0,143.0,958.0,475.0,374.0
5,1.22,13.68,0.56,300.0,357.0,1940.0,371.0,1394.0,349.0,66.0,937.0,126.0,198.0,551.0,129.0,963.0,429.0,346.0
6,1.20,12.93,0.61,328.0,428.0,1831.0,375.0,1291.0,370.0,78.0,1036.0,149.0,236.0,558.0,122.0,881.0,475.0,367.0
7,1.30,11.91,0.65,330.0,403.0,2002.0,388.0,1204.0,370.0,54.0,1031.0,139.0,225.0,665.0,141.0,813.0,488.0,361.0
8,1.16,12.67,0.51,305.0,364.0,2120.0,352.0,1342.0,355.0,74.0,985.0,138.0,214.0,533.0,118.0,874.0,443.0,350.0


It has it's own dataframe accessor:

In [10]:
pd.DataFrame.pint?

Init signature: pd.DataFrame.pint(pandas_obj)
Docstring:      <no docstring>
File:           c:\users\wil9dh\anaconda3\envs\pyrolite\lib\site-packages\pint_pandas\pint_array.py
Type:           type
Subclasses:     

To extract just the numpy array from a series:

In [11]:
units_df["FeO"].values.numpy_data

array([12.13, 13.14, 11.87, 12.7 , 11.72, 13.68, 12.93, 11.91, 12.67,
       11.75])

## Dealing with Odd Behaviour

* `np.isfinite` won't work on pint units, but `pd.isna` will - althought it needs to be mode-switched to map `np.inf` to `NaN`
* Reduction functions won't work as expected, instead a reduce-by-parts appraoch is needed in order to preserve units in arrays (possible for e.g. sum)
* Some numerical operations don't exist yet (e.g. [`.round()`](https://github.com/pandas-dev/pandas/issues/49387); arrays have no `round` method and quantities have no `rint` method)

In [12]:
def sum_by_parts(df):
    _sum = df.iloc[:, 0]
    for ix in range(1, df.columns.size):
        _sum += df.iloc[:, ix]
    return _sum


pd.set_option(
    "mode.use_inf_as_na", True
)  # to be able to use pd.isna in place of np.isfinite

In [13]:
units_df[["FeO", "MgO"]].sum(axis=1), df.std(axis=1)

c:\Users\WIL9DH\Anaconda3\envs\pyrolite\lib\site-packages\pandas\core\internals\blocks.py:343: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  res_values = np.array([[result]])


(0    0.1343
 1    0.1435
 2    0.1331
 3    0.1399
 4    0.1296
 5    0.1490
 6    0.1413
 7    0.1321
 8    0.1383
 9    0.1280
 dtype: float64,
 0    522.563633
 1    496.631414
 2    490.805472
 3    499.334931
 4    481.718363
 5    525.004720
 6    494.451279
 7    514.971511
 8    547.780418
 9    582.397405
 dtype: float64)

Apply gives indiviudal quantitiy values:

In [14]:
units_df[["FeO", "MgO"]].apply(np.sum, axis=1)

c:\Users\WIL9DH\Anaconda3\envs\pyrolite\lib\site-packages\pandas\core\dtypes\cast.py:1579: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


0    13.430000000000001 percent
1    14.350000000000001 percent
2    13.309999999999999 percent
3    13.989999999999998 percent
4                 12.96 percent
5                  14.9 percent
6    14.129999999999999 percent
7                 13.21 percent
8                 13.83 percent
9                  12.8 percent
dtype: object

Sum by parts will properly preserve array datatypes:

In [15]:
sum_by_parts(units_df[["FeO", "MgO"]])

0    13.430000000000001
1    14.350000000000001
2    13.309999999999999
3    13.989999999999998
4                 12.96
5                  14.9
6    14.129999999999999
7                 13.21
8                 13.83
9                  12.8
Name: FeO, dtype: pint[percent]

Note that processing functions which accept arrays with `pint` unit types will need to include special handling for e.g. `np.isfinite` (e.g. to convert to floats, `ratio[~np.isfinite(ratio.values.astype(float))] = np.nan`):

In [16]:
units_df.pyrochem.get_ratio("FeO/MgO")

0     9.330769230769231
1    10.859504132231406
2     8.243055555555557
3     9.844961240310079
4     9.451612903225808
5    11.213114754098362
6                10.775
7     9.161538461538461
8    10.922413793103447
9    11.190476190476188
Name: FeO/MgO, dtype: pint[dimensionless]

Other things like rounding are more difficult to get around:

In [18]:
units_df.pyrochem.convert_chemistry(to=["FeO", "MgO"]).round(2)

AttributeError: 'PintArray' object has no attribute 'round'

### Internal Array Types

Under the hood these arrays get converted to `pandas.core.arrays.numpy_.PandasArray` rather an existing as `np.ndarray`:

In [19]:
a = units_df.FeO.astype(float).values
type(a)

c:\Users\WIL9DH\Anaconda3\envs\pyrolite\lib\site-packages\pandas\core\arrays\numpy_.py:109: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result = np.asarray(scalars, dtype=dtype)  # type: ignore[arg-type]


pandas.core.arrays.numpy_.PandasArray

And extraction turns this into somethign more like a list of quantities rather than a quantity array:

In [20]:
units_df.FeO.to_numpy()

array([<Quantity(12.13, 'percent')>, <Quantity(13.14, 'percent')>,
       <Quantity(11.87, 'percent')>, <Quantity(12.7, 'percent')>,
       <Quantity(11.72, 'percent')>, <Quantity(13.68, 'percent')>,
       <Quantity(12.93, 'percent')>, <Quantity(11.91, 'percent')>,
       <Quantity(12.67, 'percent')>, <Quantity(11.75, 'percent')>],
      dtype=object)

## Uncertainties

In [21]:
import uncertainties
import uncertainties.unumpy as unp


def to_uncertain_series(s, u=np.nan):
    # uncertainties seems to reduce things to base units
    return pd.Series(
        pint_pandas.pint_array.PintArray(
            unp.uarray(s.values, s.values * u), s.pint.to_base_units().pint.units
        ).astype(s.dtype),
        index=s.index,
    )


def to_uncertain_frame(df, u=np.nan):
    return pd.DataFrame(
        {c: to_uncertain_series(df[c], u) for c in df.columns}, index=df.index
    )

In [22]:
units_df.iloc[:5, :].pyrochem.oxides

,MgO,FeO,Al2O3
0,1.3,12.13,0.6
1,1.21,13.14,0.63
2,1.44,11.87,0.65
3,1.29,12.7,0.65
4,1.24,11.72,0.65


In [23]:
udf = to_uncertain_frame(units_df, 0.01).iloc[:5,].pyrochem.oxides

In [24]:
udf

,MgO,FeO,Al2O3
0,1.300+/-0.013,12.13+/-0.12,0.600+/-0.006
1,1.210+/-0.012,13.14+/-0.13,0.630+/-0.006
2,1.440+/-0.014,11.87+/-0.12,0.650+/-0.007
3,1.290+/-0.013,12.70+/-0.13,0.650+/-0.007
4,1.240+/-0.012,11.72+/-0.12,0.650+/-0.007


In [25]:
udf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype        
---  ------  --------------  -----        
 0   MgO     5 non-null      pint[percent]
 1   FeO     5 non-null      pint[percent]
 2   Al2O3   5 non-null      pint[percent]
dtypes: pint[percent](3)
memory usage: 248.0 bytes


In [28]:
udf.pyrochem.get_ratio("FeO/MgO") .to_frame()

,FeO/MgO
0,9.33+/-0.13
1,10.86+/-0.15
2,8.24+/-0.12
3,9.84+/-0.14
4,9.45+/-0.13


## Serialization

In [29]:
udf.to_csv("udf_csv.csv", index=False)

We can see that each of the quantities is exported with its own units descriptor, but this won't necessarily be pulled back in with e.g. `df.pint.quantify()`

In [30]:
udf_csv = pd.read_csv("udf_csv.csv")
udf_csv

,MgO,FeO,Al2O3
0,1.300+/-0.013 percent,12.13+/-0.12 percent,0.600+/-0.006 percent
1,1.210+/-0.012 percent,13.14+/-0.13 percent,0.630+/-0.006 percent
2,1.440+/-0.014 percent,11.87+/-0.12 percent,0.650+/-0.007 percent
3,1.290+/-0.013 percent,12.70+/-0.13 percent,0.650+/-0.007 percent
4,1.240+/-0.012 percent,11.72+/-0.12 percent,0.650+/-0.007 percent


Importing these kinds of values is OK without uncertainties:

In [31]:
import pint

ureg = pint.UnitRegistry()
v = ureg("4.4 percent")
v

4.4 <Unit('percent')>

But this doesn't natively work with uncertainties:

In [32]:
v = ureg("4.4+/-0.1 percent")
v

DefinitionSyntaxError: missing unary operator '/'

With a bit of hacking, we can make this possible though!

In [33]:
def deserialize_unp_series(ser):
    # series will come in as an object dtype; we can split the strings
    data = ser.str.rsplit(n=1, expand=True).rename(columns={0: "arr", 1: "units"})
    assert len(set(data.units)) == 1  # assumes everything is in the same units
    return pint_pandas.pint_array.PintArray(
        unp.uarray(
            *data.arr.str.rsplit("+/-", n=1, expand=True).astype(float).T.values
        ),
        "pint[{}]".format(data.units[0]),
    )


def deserialize_unp_frame(df):
    return pd.DataFrame(
        {c: deserialize_unp_series(df[c]) for c in df.columns}, index=df.index
    )


In [34]:
deserialize_unp_frame(udf_csv).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype        
---  ------  --------------  -----        
 0   MgO     5 non-null      pint[percent]
 1   FeO     5 non-null      pint[percent]
 2   Al2O3   5 non-null      pint[percent]
dtypes: pint[percent](3)
memory usage: 248.0 bytes


What about without uncertainties?

In [35]:
units_df.iloc[:5, :].pyrochem.oxides.to_csv("pdf_csv.csv", index=False)

In [36]:
pdf_csv = pd.read_csv("pdf_csv.csv")
pdf_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   MgO     5 non-null      object
 1   FeO     5 non-null      object
 2   Al2O3   5 non-null      object
dtypes: object(3)
memory usage: 248.0+ bytes


In [37]:
pdf_csv

,MgO,FeO,Al2O3
0,1.3 percent,12.13 percent,0.6 percent
1,1.21 percent,13.14 percent,0.63 percent
2,1.44 percent,11.87 percent,0.65 percent
3,1.29 percent,12.7 percent,0.65 percent
4,1.24 percent,11.72 percent,0.65 percent


In [38]:
# we can turn the individual values into quantities easily enough
pdf_csv.applymap(ureg).iloc[0][0].magnitude


1.3

Note that wrapping a PintArray around values doesn't strip the units from the values, but we can do it manually:

In [39]:
pint_pandas.pint_array.PintArray(
    pdf_csv.iloc[:, 1].map(lambda x: ureg(x).magnitude), "pint[percent]"
)

<PintArray>
[12.13, 13.14, 11.87, 12.7, 11.72]
Length: 5, dtype: pint[percent]

The fact that PintArray doesn't strip units could lead to issues....

In [40]:
s = pdf_csv.iloc[:, 1].map(ureg)
s[1] = s[1].to("ppm")
s

0    12.13 percent
1     131400.0 ppm
2    11.87 percent
3     12.7 percent
4    11.72 percent
Name: FeO, dtype: object

In [41]:
pint_pandas.pint_array.PintArray(s, "pint[percent]").astype("pint[ppm]")

<PintArray>
[121300.00000000001 percent,           1314000000.0 ppm,
 118699.99999999999 percent,           127000.0 percent,
           117200.0 percent]
Length: 5, dtype: pint[ppm]